In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
%matplotlib inline

In [32]:
CHUNK_SIZE=2000000
OUTPUT_FILE = '../raw_data/merged3_data.csv'

le_PUZone = LabelEncoder()
le_PUBorough = LabelEncoder()
scaler1 = MinMaxScaler()
scaler2 = StandardScaler()

weather_cols = ['prcp', 'snow', 'tavg', 'tmin', 'tmax', 'wspd', 'pres']

is_first_chunk = True

In [33]:
total_rows_before = 0
total_rows_after = 0
missing_values_before = None
missing_values_after = None

pickup_datetime_min = None
pickup_datetime_max = None
dropoff_datetime_min = None
dropoff_datetime_max = None

trip_distance_stats = []
fare_amount_stats = []

for chunk in pd.read_csv('../raw_data/merged2_data.csv', chunksize=CHUNK_SIZE, parse_dates=['pickup_datetime', 'dropoff_datetime']):
    # Update stats before cleaning
    total_rows_before += len(chunk)
    missing_values_before = chunk.isnull().sum() if missing_values_before is None else missing_values_before + chunk.isnull().sum()

    if pickup_datetime_min is None or chunk['pickup_datetime'].min() < pickup_datetime_min:
        pickup_datetime_min = chunk['pickup_datetime'].min()
    if pickup_datetime_max is None or chunk['pickup_datetime'].max() > pickup_datetime_max:
        pickup_datetime_max = chunk['pickup_datetime'].max()

    if dropoff_datetime_min is None or chunk['dropoff_datetime'].min() < dropoff_datetime_min:
        dropoff_datetime_min = chunk['dropoff_datetime'].min()
    if dropoff_datetime_max is None or chunk['dropoff_datetime'].max() > dropoff_datetime_max:
        dropoff_datetime_max = chunk['dropoff_datetime'].max()

    trip_distance_stats.append(chunk['trip_distance'].describe())
    fare_amount_stats.append(chunk['fare_amount'].describe())

    # Apply preprocessing steps here
    chunk = chunk.dropna()
    chunk = chunk.drop_duplicates()
    chunk.reset_index(drop=True, inplace=True)

    chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], errors='coerce')
    chunk['dropoff_datetime'] = pd.to_datetime(chunk['dropoff_datetime'], errors='coerce')
    chunk = chunk.dropna(subset=['pickup_datetime', 'dropoff_datetime'])

    chunk['trip_duration'] = (chunk['dropoff_datetime'] - chunk['pickup_datetime']).dt.total_seconds() / 60
    chunk = chunk[(chunk['trip_duration'] > 1) & (chunk['trip_duration'] <= 120)]  # Remove unrealistic durations

    # Temporal Features
    chunk['pickup_hour'] = chunk['pickup_datetime'].dt.hour
    chunk['pickup_day'] = chunk['pickup_datetime'].dt.dayofweek
    chunk['pickup_month'] = chunk['pickup_datetime'].dt.month
    chunk['pickup_year'] = chunk['pickup_datetime'].dt.year

    # Keep only records with dates in 2021
    chunk = chunk[(chunk['pickup_datetime'].dt.year == 2021) & (chunk['dropoff_datetime'].dt.year == 2021)]
    chunk['pickup_weekend'] = chunk['pickup_day'].isin([5, 6]).astype(int)

    chunk[weather_cols] = scaler1.fit_transform(chunk[weather_cols])
    chunk['temp_variation'] = chunk['tmax'] - chunk['tmin']
    chunk[['trip_distance', 'fare_amount', 'tmin', 'tmax', 'prcp', 'snow']] = scaler2.fit_transform(chunk[['trip_distance', 'fare_amount', 'tmin', 'tmax', 'prcp', 'snow']])

    columns_to_drop = ['VendorID', 'PULocationID', 'DOLocationID', 'store_and_fwd_flag', 'RatecodeID','location_id', 'pickup_datetime', 'dropoff_datetime']
    chunk.drop(columns=columns_to_drop, inplace=True)

    if is_first_chunk:  # Fit encoders only for the first chunk
        le_PUZone.fit(chunk['PUZone'])
        le_PUBorough.fit(chunk['PUBorough'])

    chunk['PUZone'] = le_PUZone.transform(chunk['PUZone'])
    chunk['PUBorough'] = le_PUBorough.transform(chunk['PUBorough'])

    # Encoding taxi_type (yellow = 0, green = 1)
    chunk['taxi_type'] = chunk['taxi_type'].map({'yellow': 0, 'green': 1})

    # Encoding payment_type (map integer values to descriptive categories, e.g., 1 = Cash, 2 = Card, 3 = Mobile Payment)
    payment_map = {1: 'Cash', 2: 'Card', 3: 'Mobile Payment'}
    chunk['payment_type'] = chunk['payment_type'].map(payment_map)

    chunk['fare_per_mile'] = chunk['fare_amount'] / chunk['trip_distance']

    # Rolling and cyclic features
    chunk['rolling_trip_distance'] = chunk['trip_distance'].rolling(window=3).mean()
    chunk['rolling_fare_amount'] = chunk['fare_amount'].rolling(window=3).mean()
    chunk['pickup_hour_sin'] = np.sin(2 * np.pi * chunk['pickup_hour'] / 24)
    chunk['pickup_hour_cos'] = np.cos(2 * np.pi * chunk['pickup_hour'] / 24)
    chunk['pickup_day_sin'] = np.sin(2 * np.pi * chunk['pickup_day'] / 7)
    chunk['pickup_day_cos'] = np.cos(2 * np.pi * chunk['pickup_day'] / 7)

    chunk = chunk.dropna()

    chunk = chunk[(chunk['trip_distance'] > 0) & (chunk['trip_distance'] <= 50)]  # Remove extreme distances
    chunk = chunk[(chunk['fare_amount'] > 0) & (chunk['fare_amount'] <= 500)]  # Remove extreme fares

    # Write Processed Chunk to CSV
    if is_first_chunk:
        chunk.to_csv(OUTPUT_FILE, index=False, mode='w')  # Write with headers for the first chunk
        is_first_chunk = False
    else:
        chunk.to_csv(OUTPUT_FILE, index=False, header=False, mode='a')  # Append without headers


    # Update stats after cleaning
    total_rows_after += len(chunk)
    missing_values_after = chunk.isnull().sum() if missing_values_after is None else missing_values_after + chunk.isnull().sum()

# Compute final stats
print("\n=== Final Net Stats ===")
print("Total rows before cleaning:", total_rows_before)
print("Total rows after cleaning:", total_rows_after)
print("Total missing values before cleaning:")
print(missing_values_before)
print("Total missing values after cleaning:")
print(missing_values_after)
print("Pickup Datetime Range:", pickup_datetime_min, "to", pickup_datetime_max)
print("Dropoff Datetime Range:", dropoff_datetime_min, "to", dropoff_datetime_max)

# Combine trip distance and fare stats across chunks
combined_trip_distance_stats = pd.concat(trip_distance_stats, axis=1).mean(axis=1)
combined_fare_amount_stats = pd.concat(fare_amount_stats, axis=1).mean(axis=1)

print("\nTrip Distance Stats (combined across all chunks):")
print(combined_trip_distance_stats)
print("\nFare Amount Stats (combined across all chunks):")
print(combined_fare_amount_stats)

/tmp/ipykernel_68070/1395817498.py:14: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../raw_data/merged2_data.csv', chunksize=CHUNK_SIZE, parse_dates=['pickup_datetime', 'dropoff_datetime']):
/tmp/ipykernel_68070/1395817498.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv('../raw_data/merged2_data.csv', chunksize=CHUNK_SIZE, parse_dates=['pickup_datetime', 'dropoff_datetime']):
/tmp/ipykernel_68070/1395817498.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv('../raw_data/merged2_data.csv', chunksize=CHUNK_SIZE, parse_dates=['pickup_datetime', 'dropoff_datetime']):
/tmp/ipy


=== Final Net Stats ===
Total rows before cleaning: 29866438
Total rows after cleaning: 105
Total missing values before cleaning:
VendorID                   0
pickup_datetime            0
dropoff_datetime           0
store_and_fwd_flag         0
RatecodeID                 0
PULocationID               0
DOLocationID               0
passenger_count            0
trip_distance              0
fare_amount                0
extra                      0
mta_tax                    0
tip_amount                 0
tolls_amount               0
improvement_surcharge      0
total_amount               0
payment_type               0
congestion_surcharge       0
taxi_type                  0
date                       0
tavg                     505
tmin                     505
tmax                     505
prcp                     505
snow                     505
wdir                     505
wspd                     505
pres                     505
location_id                0
PUBorough                  0